<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#CTR-prediction" data-toc-modified-id="CTR-prediction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>CTR-prediction</a></span><ul class="toc-item"><li><span><a href="#Problem-Formulation" data-toc-modified-id="Problem-Formulation-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Problem Formulation</a></span></li><li><span><a href="#Dataset" data-toc-modified-id="Dataset-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Dataset</a></span><ul class="toc-item"><li><span><a href="#Dataset-construction:" data-toc-modified-id="Dataset-construction:-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Dataset construction:</a></span></li><li><span><a href="#Format:" data-toc-modified-id="Format:-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Format:</a></span></li></ul></li><li><span><a href="#Metrics" data-toc-modified-id="Metrics-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Metrics</a></span></li></ul></li><li><span><a href="#Dataset-preprocessing" data-toc-modified-id="Dataset-preprocessing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Dataset preprocessing</a></span><ul class="toc-item"><li><span><a href="#ML-Pipelines-(Transformers,-Estimators)" data-toc-modified-id="ML-Pipelines-(Transformers,-Estimators)-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-pipeline.html#pipeline-components" target="_blank">ML Pipelines (Transformers, Estimators)</a></a></span><ul class="toc-item"><li><span><a href="#Prepare-stages-of-pipeline" data-toc-modified-id="Prepare-stages-of-pipeline-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Prepare stages of pipeline</a></span></li><li><span><a href="#Fit-and-save-pipeline" data-toc-modified-id="Fit-and-save-pipeline-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Fit and save pipeline</a></span></li><li><span><a href="#Load-fitted-pipeline" data-toc-modified-id="Load-fitted-pipeline-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Load fitted pipeline</a></span></li><li><span><a href="#Transform-dataset-using-pipeline" data-toc-modified-id="Transform-dataset-using-pipeline-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Transform dataset using pipeline</a></span></li><li><span><a href="#Make-dataset-split" data-toc-modified-id="Make-dataset-split-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>Make dataset split</a></span></li></ul></li></ul></li><li><span><a href="#Classification" data-toc-modified-id="Classification-3"><span class="toc-item-num">3&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-classification-regression.html" target="_blank">Classification</a></a></span><ul class="toc-item"><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-classification-regression.html#logistic-regression" target="_blank">Logistic Regression</a></a></span><ul class="toc-item"><li><span><a href="#Define-and-Train-model" data-toc-modified-id="Define-and-Train-model-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Define and Train model</a></span></li></ul></li></ul></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html" target="_blank">Evaluation</a></a></span><ul class="toc-item"><li><span><a href="#Binary-classification-metrics" data-toc-modified-id="Binary-classification-metrics-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html#binary-classification" target="_blank">Binary classification metrics</a></a></span></li><li><span><a href="#Make-submission" data-toc-modified-id="Make-submission-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Make submission</a></span></li></ul></li></ul></div>

# CTR-prediction

## Problem Formulation

$\newcommand{\vecw}{{\bf w}}$
$\newcommand{\vecx}{{\bf x}}$

* Dataset: $X^N = \{ z_i \}^N_{i=1}$, где $z_i = (\vecx_i, y_i) \sim P(z), y_i \in \{0,1\}$
* Prediction: $$ \hat{y}_i = f_{\vecw}(\vecx_i) =  \mathbb{P} \left\{ y = 1 \mid \vecx_i \right\} $$
* Loss function (Binary Cross-Entropy): $$ \min\limits_{\vecw} \quad \frac{\lambda}{2}\| \vecw \|^2_2 - \frac{1}{N} \sum\limits_{i=1}^{N} y_i \log \hat{y}_i + (1-y_i) \log(1-\hat{y}_i) $$

## Dataset
$ $
<details>
  <summary>Click here to see the details</summary>

For more details see `/data/criteo/readme.txt`

### Dataset construction:


>There are 13 features taking **integer** values and 26
**categorical** features. The values of the categorical features have been hashed
onto 32 bits for anonymization purposes. 
Some features may have missing values.

> The rows are chronologically ordered by `id` column.

> The test set corresponds to events on the day following the training period. 
The first column (`label`) has been removed.


### Format:

> The columns are comma separeted with the following schema:
`<label>,<integer feature 1>, ... <integer feature 13>,<categorical feature 1>, ... <categorical feature 26>,<id>`

> When a value is missing, the field is "". There is no `label` field in the test set.

</details>
    
## Metrics

The evaluation metrics for this task are
* ROC AUC
* LogLoss
* [Normalized Entropy](https://quinonero.net/Publications/predicting-clicks-facebook.pdf)

In [1]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import os
import sys
import glob
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import pyspark
import pyspark.sql.functions as F
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import Row


spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("spark_sql_examples") \
    .config("spark.executor.memory", "6g") \
    .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [2]:
DATA_PATH = '/workspace/data/criteo'

TRAIN_PATH = os.path.join(DATA_PATH, 'train.csv')

Lets begin our introduction to Spark [MLlib](https://spark.apache.org/docs/latest/ml-guide.html)

---
# Dataset preprocessing

Before we can train any prediction model on our dataset we need to conver each row into real-valued features vector ($\vecx \in \mathbb{R}^n$).

Spark MLlib provides easy to use tools for preprocessing raw features and turning them into suitable format.

In [3]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TRAIN_PATH)

In [4]:
df = df.sample(False, 0.5)

In [5]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (

Keeping only first two categorical features for simplicity

In [6]:
num_columns = ['_c{}'.format(i) for i in range(1, 14)]
cat_columns = ['_c{}'.format(i) for i in range(14, 40)][:2]
len(num_columns), len(cat_columns)

(13, 2)

In [7]:
df = df.fillna(0, subset=num_columns)

## [ML Pipelines (Transformers, Estimators)](https://spark.apache.org/docs/latest/ml-pipeline.html#pipeline-components)


MLlib standardizes APIs for machine learning algorithms to make it easier to combine multiple algorithms into a single pipeline, or workflow.

* `Transformer`: A Transformer is an algorithm which can transform one DataFrame into another DataFrame. E.g., an ML model is a Transformer which transforms a DataFrame with features into a DataFrame with predictions.


* `Estimator`: An Estimator is an algorithm which can be fit on a DataFrame to produce a Transformer. E.g., a learning algorithm is an Estimator which trains on a DataFrame and produces a model.


* `Pipeline`: A Pipeline chains multiple Transformers and Estimators together to specify an ML workflow.

---
Basically speaking `transformer` is an instance of class that implements `transform` method, and both `estimator` and `pipeline` implements `transform` and `fit` methods.

---

### Prepare stages of pipeline

We might benefit from using `StringIndexer, OneHotEncoderEstimator, VectorAssembler` (see [doc](https://spark.apache.org/docs/latest/ml-features) for details) 

In [8]:
print("count", df.count())
print(df.limit(20).toPandas())

count 1833503
    _c0  _c1   _c2  _c3  _c4    _c5  _c6  _c7  _c8  _c9  ...      _c31  \
0     1    0    -1    0    0   1465    0   17    0    4  ...  e5f8f18f   
1     1    0     1   20   16   1548   93   42   32  912  ...  1f868fdd   
2     0    0    53    0   10   6550   98   34   11  349  ...  fa0643ee   
3     0    0     7    1    0   9128    0    0    0    7  ...  d6be9f7b   
4     0    0   179    4    4  17232    0    0   18    7  ...  2804effd   
5     0    0    18   15    9   4494    0    0    9    8  ...  e4ca448c   
6     1    1     5   33    6    375   74    3   46  258  ...  04d863d5   
7     0    0    19   31    8   7130  126    1   20  116  ...  df4fffb7   
8     0    0     8    0    2  30617    0    0    5   40  ...  281769c2   
9     0    0  2242   13    7  11786   63   29   13   87  ...  bc48b783   
10    0    0     1    0    1      0    0    0    1    1  ...  19fec6cc   
11    0    0     0    0    3   1992    5    6    5    5  ...  e7e991cb   
12    1    0     0    3 

In [9]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler

indexersTF = [StringIndexer(inputCol=column, outputCol=column+'_index').setHandleInvalid("keep").fit(df) for column in cat_columns]

cat_columns_index = list(map(lambda x : x + '_index', cat_columns))
cat_columns_vecs = list(map(lambda x: x + '_vec', cat_columns))
oneHotTF = OneHotEncoderEstimator(inputCols=cat_columns_index, outputCols=cat_columns_vecs)
vectorAssemblerTF = VectorAssembler(inputCols=num_columns + cat_columns_vecs, outputCol='features')

pipeline = Pipeline(stages=indexersTF + [oneHotTF, vectorAssemblerTF])




### Fit and save pipeline

In [30]:
transform_model = pipeline.fit(df)

# transform_model.save('transform_model')


### Load fitted pipeline

In [11]:
from pyspark.ml import PipelineModel

transform_model_loaded = PipelineModel.load('transform_model')

### Transform dataset using pipeline

In [12]:
df_transformed = transform_model \
    .transform(df) \
    .withColumnRenamed('_c0', 'label') \
    .select('features', 'label', 'id')

print(df_transformed.limit(20).toPandas())


                                             features  label   id
0   (0.0, -1.0, 0.0, 0.0, 1465.0, 0.0, 17.0, 0.0, ...      1   12
1   (0.0, 1.0, 20.0, 16.0, 1548.0, 93.0, 42.0, 32....      1   26
2   (0.0, 53.0, 0.0, 10.0, 6550.0, 98.0, 34.0, 11....      0   85
3   (0.0, 7.0, 1.0, 0.0, 9128.0, 0.0, 0.0, 0.0, 7....      0  139
4   (0.0, 179.0, 4.0, 4.0, 17232.0, 0.0, 0.0, 18.0...      0  151
5   (0.0, 18.0, 15.0, 9.0, 4494.0, 0.0, 0.0, 9.0, ...      0  175
6   (1.0, 5.0, 33.0, 6.0, 375.0, 74.0, 3.0, 46.0, ...      1  212
7   (0.0, 19.0, 31.0, 8.0, 7130.0, 126.0, 1.0, 20....      0  252
8   (0.0, 8.0, 0.0, 2.0, 30617.0, 0.0, 0.0, 5.0, 4...      0  255
9   (0.0, 2242.0, 13.0, 7.0, 11786.0, 63.0, 29.0, ...      0  268
10  (0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, ...      0  348
11  (0.0, 0.0, 0.0, 3.0, 1992.0, 5.0, 6.0, 5.0, 5....      0  376
12  (0.0, 0.0, 3.0, 6.0, 8223.0, 123.0, 5.0, 10.0,...      1  382
13  (0.0, -1.0, 0.0, 0.0, 8020.0, 26.0, 6.0, 0.0, ...      1  405
14  (1.0, 

### Make dataset split

Spark provides [randomSplit](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.randomSplit) method.

It is not the best choice in our task since we have chronological order in data.

We need to implement our own split function which will split the data in parts with respect to chronological order.

In [13]:
from pyspark.sql.window import Window

def split_by_col(df, split_col, parts_fractions):
    """
    df - DataFrame
    split_col - total order column
    parts_fractions - fractions of resulting parts
    """
    
    split_col_window = Window.orderBy(split_col)

    df_percent_ranks = df.withColumn('rank', F.percent_rank().over(split_col_window))
    
    parts = []
    start_fraction = 0
    for part_fraction in parts_fractions:
        part = df_percent_ranks \
            .filter(F.col('rank') >= start_fraction) \
            .filter(F.col('rank') < start_fraction + part_fraction) \
            .drop('rank')
        parts.append(part)
        start_fraction += part_fraction

    return parts

In [14]:
train_df, val_df, test_df = split_by_col(df_transformed, 'id', [0.8, 0.1, 0.1])

In [15]:
N = df.count()

In [16]:
train_df.count() / N, val_df.count() / N, test_df.count() / N

(0.7999997818383717, 0.09999983637877877, 0.09999983637877877)

---
# [Classification](https://spark.apache.org/docs/latest/ml-classification-regression.html)

## [Logistic Regression](https://spark.apache.org/docs/latest/ml-classification-regression.html#logistic-regression)

### Define and Train model

In [17]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(regParam=0.3)
lr_model = lr.fit(train_df)

In [18]:
lr_model.transform(val_df).first()

Row(features=SparseVector(1958, {0: 5.0, 1: 156.0, 3: 3.0, 4: 210.0, 5: 6.0, 6: 5.0, 7: 5.0, 8: 6.0, 9: 1.0, 10: 1.0, 12: 3.0, 14: 1.0, 1404: 1.0}), label=0, id=455266596465, rawPrediction=DenseVector([0.7734, -0.7734]), probability=DenseVector([0.6842, 0.3158]), prediction=0.0)

---
# [Evaluation](https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html)

## [Binary classification metrics](https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html#binary-classification)

* ROC AUC
* LogLoss
* Normalized Entropy

In [19]:
from pyspark.sql.types import *
from pyspark.mllib.evaluation import BinaryClassificationMetrics
import math
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType


def rocauc(model, df):
    pred = model.transform(df)
    predictionAndLabels = pred.rdd.map(lambda x: (float(x.probability[1]), float(x.label)))
    metrics = BinaryClassificationMetrics(predictionAndLabels)
    return metrics.areaUnderROC


def logloss(model, df):
    pred = model.transform(df)

    firstelementUdf = udf(lambda v:float(v[1]), FloatType())
    df_logloss = pred \
        .withColumn(
            'logloss'
            , -F.col('label') * F.log(firstelementUdf('probability')) - (1. - F.col('label')) * F.log(1. - (firstelementUdf('probability')))
        )

    res_value = float(df_logloss \
        .agg(F.mean('logloss')) \
        .head()[0])


    return res_value


def ne(model, df):
    ll = logloss(model, df)
    p = df.agg(F.avg(F.col('label'))).head()[0]
    const_entropy = -p * math.log(p) - (1. - p) * math.log(1. - p)
    return ll / const_entropy


In [20]:
rocauc(lr_model, val_df)

0.6991721429752401

In [21]:
ne(lr_model, val_df)

0.9452191045321535

In [22]:
rocauc(lr_model, test_df)

0.7008679800202761

In [23]:
ne(lr_model, test_df)

0.9445117066566621

## Make submission

Join the [competition](https://www.kaggle.com/c/mlbd-20-ctr-prediction-1) and make a submission

In [24]:
TEST_PATH = os.path.join(DATA_PATH, 'test.csv')

df_test = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TEST_PATH)

df_test.printSchema()

df_test = df_test.fillna(0, subset=num_columns)

root
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (nullable = true)
 |-- _c28: string (

In [25]:
df_test_transformed = transform_model \
    .transform(df_test) \
    .select('features', 'id') \

print(df_test_transformed.limit(20).toPandas())


                                             features            id
0   (0.0, 19.0, 2.0, 4.0, 4576.0, 6.0, 6.0, 5.0, 1...  566935904713
1   (0.0, 1.0, 1.0, 0.0, 5688.0, 0.0, 0.0, 2.0, 10...  566935904715
2   (0.0, 445.0, 2.0, 2.0, 8579.0, 26.0, 1.0, 2.0,...  566935904727
3   (0.0, 172.0, 7.0, 1.0, 2008.0, 143.0, 24.0, 28...  566935904737
4   (0.0, 11.0, 4.0, 4.0, 14.0, 0.0, 0.0, 4.0, 6.0...  566935904741
5   (1.0, 0.0, 19.0, 27.0, 108.0, 33.0, 14.0, 33.0...  566935904745
6   (7.0, 2.0, 2.0, 6.0, 296.0, 13.0, 27.0, 9.0, 2...  566935904752
7   (0.0, 0.0, 210.0, 46.0, 16102.0, 0.0, 0.0, 46....  566935904759
8   (0.0, 2.0, 1.0, 1.0, 17621.0, 40.0, 3.0, 5.0, ...  566935904767
9   (0.0, 56.0, 0.0, 0.0, 9794.0, 0.0, 0.0, 30.0, ...  566935904768
10  (0.0, 12.0, 1.0, 3.0, 6081.0, 69.0, 8.0, 6.0, ...  566935904778
11  (4.0, 3.0, 9.0, 11.0, 1229.0, 53.0, 4.0, 43.0,...  566935904785
12  (0.0, 1124.0, 0.0, 9.0, 2756.0, 13.0, 42.0, 11...  566935904789
13  (1.0, 0.0, 4.0, 1.0, 1284.0, 13.0, 1.0, 10.0

In [33]:
firstelementUdf = udf(lambda v:float(v[1]), FloatType())

pred_test_df = lr_model \
        .transform(df_test_transformed) \
        .select('id', 'probability') \
        .withColumn('proba', firstelementUdf('probability')) \
        .drop('probability')

print(pred_test_df.take(10))
pred_test_df.show(5)

[Row(id=566935904713, proba=0.24282529950141907), Row(id=566935904715, proba=0.2642812728881836), Row(id=566935904727, proba=0.26935726404190063), Row(id=566935904737, proba=0.35397103428840637), Row(id=566935904741, proba=0.25624197721481323), Row(id=566935904745, proba=0.2616666853427887), Row(id=566935904752, proba=0.3720690608024597), Row(id=566935904759, proba=0.15140342712402344), Row(id=566935904767, proba=0.3080241084098816), Row(id=566935904768, proba=0.21472062170505524)]
+------------+----------+
|          id|     proba|
+------------+----------+
|566935904713| 0.2428253|
|566935904715|0.26428127|
|566935904727|0.26935726|
|566935904737|0.35397103|
|566935904741|0.25624198|
+------------+----------+
only showing top 5 rows



In [34]:
TEST_PREDICTIONS_PATH = 'test_predictions'

pred_test_df \
    .coalesce(1) \
    .write \
    .option("header", "true") \
    .mode('overwrite') \
    .csv('/workspace/data/' + TEST_PREDICTIONS_PATH)
